### GRAPHSA模块展示图信息
### HISTOGRAMS模块展示训练过程
### SCALARS模块展示loss等训练信息

In [40]:
import tensorflow as tf
import numpy as np
from keras import backend as K
K.clear_session()

def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):  
    '''层'''
    layer_name = 'layer%s' % n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')  
            tf.summary.histogram(layer_name+'/weights', Weights) # Weights用histogram展示
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')   
            tf.summary.histogram(layer_name+'/biases', biases)  # biase用histogram展示
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+'/outputs', outputs)   # outputs用histogram展示
        return outputs

# 数据
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

# 定义传入值
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None,1], name='x_input') 
    ys = tf.placeholder(tf.float32, [None,1], name='y_input')

l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)   # 隐藏层
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)   # 输出层

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))# 损失
    tf.summary.scalar('loss', loss)                 # loss使用scalar_summary()定义展示
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  # 优化器

sess = tf.Session()
merged = tf.summary.merge_all()  # 合并展示信息
writer = tf.summary.FileWriter('logs/', sess.graph)   # 将图信息存在文件中

sess.run(tf.initialize_all_variables())

for i in range(1000):
    sess.run(train_step, feed_dict={xs:x_data, ys:y_data})  # 用全部的数据来运算
    if i%50==0:
        result = sess.run(merged, feed_dict={xs:x_data, ys:y_data})   # run merged
        writer.add_summary(result, i)                                 # 记录

#### TensorFlow报错：AttributeError: module 'tensorflow._api.v1.train' has no attribute 'SummaryWriter'等

**分析**：版本更新，方法调用方式改变

**解决方式**：

| 报错                                                         | 原方法                   | 更改后方法              |
| ------------------------------------------------------------ | ------------------------ | ----------------------- |
| AttributeError: module 'tensorflow._api.v1.train' has no attribute 'SummaryWriter' | tf.train.SummaryWriter() | tf.summary.FileWriter() |
| AttributeError: module 'tensorflow' has no attribute 'merge_all_summaries' | tf.merge_all_summaries() | tf.summary.merge_all()  |
| AttributeError: module 'tensorflow' has no attribute 'histogram_summary' | tf.histogram_summary()   | tf.summary.histogram()  |
| AttributeError: module 'tensorflow' has no attribute 'scalar_summary' | tf.scalar_summary()      | tf.summary.scalar()     |
| AttributeError: module 'tensorflow' has no attribute 'image_summary' | tf.image_summary()       | tf.summary.image()      |



#### TensorFlow报错：InvalidArgumentError: You must feed a value for placeholder tensor 'inputs/y_input' with dtype float and shape \[?,1]

Jupyter Notebook 中使用 TensorFlow 构建神经网络时，通过 tensotboard 可视化数据变化趋势，run tf.summary.merge_all() 操作时**报错**：

![图1](E:\博客\TensorFlow\HISTOGRAMS_SCALARS\图1.PNG)

> InvalidArgumentError: You must feed a value for placeholder tensor 'inputs/y_input' with dtype float and shape \[?,1]
> 		[[{{node inputs/y_input}}]]

**分析**：jupyter 会记住会话中创建的 summary，影响后序操作

**解决方式1**：删除之前的生成的文件

**解决方式2**：显示合并 summary，而不是使用 tf.summary.merge_all()，如下所示

```python
weights_summary = tf.summary.histogram("weights", W)
biases_summary = tf.summary.histogram("biases", b)
y_summary = tf.summary.histogram("y", y)

merged = tf.summary.merge([weights_summary, biases_summary, y_summary])
```

**解决方式3**：设置默认图

```python
with tf.Graph().as_default() as g:
	# your tf operations code
```

或

```python
tf.reset_default_graph()
# your tf operations code
```

**解决方式4**：使用 keras 模块的 backend.clear_session() 方法

- 安装 keras，cmd 窗口下输入如下命令：

  > pip install keras

- 构建神经网络时，调用如下方法：

  ```python
  from keras import backend as K
  K.clear_session()
  # your tf operations code
  ```



**参考**：

[https://blog.csdn.net/m0_37870649/article/details/79428960](https://blog.csdn.net/m0_37870649/article/details/79428960)

[https://stackoverflow.com/questions/35114376/error-when-computing-summaries-in-tensorflow](https://stackoverflow.com/questions/35114376/error-when-computing-summaries-in-tensorflow)
